## Monte Carlo Simulation

For today's session, we will no longer be performing simulations using molecular dynamics. Instead, we will be using the second of the "Big 2" simulation approaches: Monte Carlo.

Monte Carlo (MC) offers several advantages and disadvantages over molecular dynamics (MD), and it will depend on the system and problem of interest as to which simulation approach is appropriate. For systems where large energy barriers can result in systems that are easily trapped in local minima, MC can provide a significant advantage over MD. However, typically there is no "time" in a Monte Carlo simulation, thus properties that require time-averaging can no be assessed via MC.

In this session, you will introduced to the basics of MC through simulations of a simple Lennard-Jones system (comparable to the system you performed MD simulations of on Tuesday). While several open-source, highly-optimized MD codes exist (you have already been introduced to HOOMD and GROMACS), relatively few MC codes exist. As a result, for this session we will be using an ad hoc MC code written in C++, that has been wrapped in Python for clarity. We will explore some of the underlying code and observe how changing various run parameters effect the result of the simulation.

### Importing the MC Code

The MC code is contained in several files located in the `src` directory. Modules exist for the LJ force field, the LJ system, and the Monte Carlo algorithm. We will import these three classes here.

In [ ]:
from src import ForceField, LJBox, MonteCarlo

### Define the force field

Let's define the Lennard-Jones parameters that will dictate how our particles will interact with one another. In the box below we will define values for `sigma`, `epsilon`, and `cutoff`. Recall these parameters from the Lennard-Jones equation:

\begin{align}
U_{LJ} = 4 \epsilon \left [ {\left ( \frac{\sigma}{r} \right )} ^{12} - {\left ( \frac{\sigma}{r} \right )} ^{6} \right ],\: r \le r_{cut}
\end{align}

We will begin by using a sigma of 1.0, an epsilon of 1.0, and a cutoff of 2.5.

In [ ]:
sigma = 1.0 
epsilon = 1.0 
cutoff = 2.5 

forcefield = ForceField(sigma=sigma, epsilon=epsilon, cutoff=cutoff)

For this code, force field parameters are stored within a `ForceField` object. Let's take a quick look at this (simple) class.

In [ ]:
!cat src/forcefield.py

As we can see above, the `ForceField` class simply features a constructor to load force field information into several class attributes and a `show` method to plot the force field.

Let's use the `show` method here to obtain a better appreciation for the force field we've defined.

In [ ]:
forcefield.show()

#### Define system parameters

Now we want to define our system parameters. For this system these parameters will be the total number of particles and the number density of particles to place in the box.

We will begin by defining a system of 216 particles and a number density of 0.7.

In [ ]:
n_particles = 216 
number_density = 0.7

The `LJBox` class will take these system parameters and use mBuild to create the system. Let's take a quick look at that class.

In [ ]:
!cat src/ljbox.py

In [ ]:
system = LJBox(n_particles, number_density)

#### Define Monte Carlo parameters

Now let's define the parameters for our Monte Carlo simulation. There are several parameters that we need to define:
  - Temperature &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; Recall $\beta = 1 / {k_B T}$
  - $\Delta_{x, init}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp; Initial maximum particle displacement
  - Acceptance probability &nbsp; &nbsp; &nbsp; At what probability do we want move to be accepted?
  - $n_{relax}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Number of steps to relax from the initial configuration
  - $n_{MC}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp; Number of MC steps to perform after the relaxation period

In [ ]:
temperature = 1.2
dx = 0.1            # Initial maximum displacement
target = 0.5        # Target acceptance probabality
n_relax = 10000     # Number of steps to relax from initial configuration
n_mc = 100000       # Total number of MC steps

We will feed these parameters into the constructor for a `MonteCarlo` object. This class provides a wrapper around the C++ code that performs the Monte Carlo simulation.

In [ ]:
mc = MonteCarlo(forcefield=forcefield,
                system=system,
                dx=dx,
                temperature=temperature,
                target=target)

#### Relax the system

First we will perform a relaxation, both to move our system out of a high energy initial state, and to determine the optimum value of `dx` to yield the target acceptance probability.

In [ ]:
mc.relax(n_relax, adjust_freq=100)

Let's view the trajectory of the relaxation.

In [ ]:
import nglview as nv
import mdtraj as md
traj = md.load('relax.xyz', top='system_init.gro')
nv.show_mdtraj(traj, representations=[{'type': 'spacefill', 'params': {'radius': 0.5}}])

#### Perform production Monte Carlo run

Now that we have optimized `dx` we can perform a production run.

In [ ]:
mc.run(n_mc)

#### View the trajectory

In [ ]:
import nglview as nv
import mdtraj as md
traj = md.load('run.xyz', top='system_init.gro')
nv.show_mdtraj(traj, representations=[{'type': 'spacefill', 'params': {'radius': 0.5}}])